In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')

In [2]:
housing_prepared = np.loadtxt('datasets/housing/housing_prepared.txt', dtype=float)
housing_labels = pd.read_csv("datasets/housing/housing_labels.csv")

# 모델 훈련

## LinearRegression

In [3]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

### MSE

In [4]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68644.88938931034

### MAE

In [5]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
lin_mae

49519.3445373062

## DecisionTreeRegressor

In [6]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(random_state=42)

In [7]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

# 교차검증을 통한 평가

## DecisionTreeRegressor

In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [9]:
def display_scores(scores):
    print("점수:", scores)
    print("평균:", scores.mean())
    print("표준편차:", scores.std())

display_scores(tree_rmse_scores)

점수: [72845.67443449 70103.03601583 69431.52687513 72671.66704237
 68906.7738104  79131.58707634 70344.17934657 73035.60861677
 70060.07148353 71273.59955054]
평균: 71780.37242519669
표준편차: 2818.0180214479633


## LinearRegression

In [10]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

점수: [71762.76302789 64122.05022719 67760.87417175 68635.19126601
 66987.90744008 72527.15803886 74093.20819009 68802.33757141
 66485.9695096  70146.08293189]
평균: 69132.35423747802
표준편차: 2881.3825001317587


## RandomForestRegressor

In [11]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(n_estimators=10, random_state=42)

In [12]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

22366.609160370783

In [13]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

점수: [53538.55469313 50612.72636093 48883.37258017 53884.28790649
 50847.38317638 54857.41426501 56030.79800034 51960.32206005
 51487.92311347 55700.72774435]
평균: 52780.35099003227
표준편차: 2261.989537187567


In [14]:
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

count       10.000000
mean     69132.354237
std       3037.243837
min      64122.050227
25%      67181.149123
50%      68718.764419
75%      71358.593004
max      74093.208190
dtype: float64

## SVR

In [15]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

111095.06688520181

# 모델 세부 튜닝

## 그리드 탐색

In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # 하이퍼파라미터 12(=3×4)개의 조합을 시도합니다.
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # bootstrap은 False로 하고 6(=2×3)개의 조합을 시도합니다.
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# 다섯 폴드에서 훈련하면 총 (12+6)*5=90번의 훈련이 일어납니다.
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [17]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [18]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

In [19]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

63916.575761905464 {'max_features': 2, 'n_estimators': 3}
54936.41801736748 {'max_features': 2, 'n_estimators': 10}
52917.16736317768 {'max_features': 2, 'n_estimators': 30}
60172.20212674653 {'max_features': 4, 'n_estimators': 3}
52570.515034035256 {'max_features': 4, 'n_estimators': 10}
50244.652386918846 {'max_features': 4, 'n_estimators': 30}
58082.56227380028 {'max_features': 6, 'n_estimators': 3}
51522.85913933991 {'max_features': 6, 'n_estimators': 10}
49970.30111162011 {'max_features': 6, 'n_estimators': 30}
58863.984741627086 {'max_features': 8, 'n_estimators': 3}
52327.82463997598 {'max_features': 8, 'n_estimators': 10}
49898.3288291329 {'max_features': 8, 'n_estimators': 30}
62379.30604451122 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54463.167358450715 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59894.3616401629 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52729.78518142424 {'bootstrap': False, 'max_features': 3, 'n_estimators':

In [20]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.111631,0.004031,0.007234,0.001189,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-4.131778e+09,-3.727541e+09,...,-4.085329e+09,1.866210e+08,18,-1.154993e+09,-1.089537e+09,-1.154284e+09,-1.119194e+09,-1.090311e+09,-1.121664e+09,2.896956e+07
1,0.377681,0.008253,0.015445,0.002839,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-2.973630e+09,-2.807277e+09,...,-3.018010e+09,1.173140e+08,11,-5.983790e+08,-5.908300e+08,-6.118786e+08,-5.723156e+08,-5.908458e+08,-5.928498e+08,1.282511e+07
2,1.100389,0.020411,0.056879,0.005814,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-2.806410e+09,-2.672936e+09,...,-2.800227e+09,8.028030e+07,9,-4.413960e+08,-4.326514e+08,-4.538088e+08,-4.312895e+08,-4.317327e+08,-4.381757e+08,8.651154e+06
3,0.184915,0.007735,0.008026,0.000867,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-3.590167e+09,-3.500472e+09,...,-3.620694e+09,1.271107e+08,16,-9.781426e+08,-9.829645e+08,-1.004610e+09,-1.016225e+09,-1.005646e+09,-9.975178e+08,1.451583e+07
4,0.551928,0.022027,0.021461,0.001581,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-2.765209e+09,-2.615408e+09,...,-2.763659e+09,1.165172e+08,7,-5.065902e+08,-5.257122e+08,-5.080925e+08,-5.180840e+08,-5.279390e+08,-5.172836e+08,8.764054e+06
5,1.736577,0.062862,0.050885,0.004489,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-2.532510e+09,-2.439557e+09,...,-2.524525e+09,8.447457e+07,3,-3.778335e+08,-3.900195e+08,-3.881955e+08,-3.836671e+08,-3.894466e+08,-3.858324e+08,4.579274e+06
6,0.248764,0.014671,0.008911,0.002056,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-3.361951e+09,-3.305042e+09,...,-3.373584e+09,1.374572e+08,13,-9.052984e+08,-9.570919e+08,-8.994493e+08,-8.944361e+08,-9.152935e+08,-9.143138e+08,2.248678e+07
7,0.734047,0.036090,0.019111,0.004799,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-2.626441e+09,-2.663824e+09,...,-2.654605e+09,6.719470e+07,5,-5.002532e+08,-5.147184e+08,-5.013538e+08,-4.954151e+08,-5.153067e+08,-5.054094e+08,8.093635e+06
8,2.307468,0.083373,0.055755,0.004942,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-2.454199e+09,-2.444403e+09,...,-2.497031e+09,7.085197e+07,2,-3.776175e+08,-3.872179e+08,-3.881859e+08,-3.762770e+08,-3.857195e+08,-3.830036e+08,5.024957e+06
9,0.329387,0.029554,0.007666,0.001131,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",-3.602004e+09,-3.210567e+09,...,-3.464969e+09,1.385385e+08,14,-9.486041e+08,-9.160713e+08,-9.030090e+08,-9.078063e+08,-9.450896e+08,-9.241161e+08,1.905668e+07


## 랜덤 탐색

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', 
                                random_state=42, n_jobs=-1)
rnd_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A1355FAA00>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A13560CB20>},
                   random_state=42, scoring='neg_mean_squared_error')

In [22]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

49131.83457958955 {'max_features': 7, 'n_estimators': 180}
51450.84815656001 {'max_features': 5, 'n_estimators': 15}
50686.3351464383 {'max_features': 3, 'n_estimators': 72}
50797.95310722897 {'max_features': 5, 'n_estimators': 21}
49183.94670013987 {'max_features': 7, 'n_estimators': 122}
50650.96405423256 {'max_features': 3, 'n_estimators': 75}
50509.5936967293 {'max_features': 3, 'n_estimators': 88}
49532.46366377122 {'max_features': 5, 'n_estimators': 100}
50300.919499325304 {'max_features': 3, 'n_estimators': 150}
65178.44470135641 {'max_features': 5, 'n_estimators': 2}


## 최상의 모델과 오차 분석

In [23]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([6.95556703e-02, 6.03855087e-02, 4.21345961e-02, 1.52923840e-02,
       1.54474318e-02, 1.58998048e-02, 1.49720520e-02, 3.79203496e-01,
       5.48287342e-02, 1.07034780e-01, 4.80223643e-02, 6.82078731e-03,
       1.65774711e-01, 7.83480660e-05, 1.52165360e-03, 3.02767843e-03])

In [61]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
# cat_encoder = cat_pipeline.named_steps["cat_encoder"]
cat_encoder = full_pipeline.named_transformers_["cat_encoder"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

## 테스트 세트로 시스템 평가하기

In [39]:
strat_train_set = pd.read_csv("datasets/housing/strat_train_set.csv")
strat_test_set = pd.read_csv("datasets/housing/strat_test_set.csv")

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

In [ ]:
from scipy import stats

In [ ]:
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
mean = squared_errors.mean()
m = len(squared_errors)

np.sqrt(stats.t.interval(confidence, m - 1,
                         loc=np.mean(squared_errors),
                         scale=stats.sem(squared_errors)))

In [ ]:
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

In [ ]:
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)

# 추가 내용

## 전처리와 예측을 포함한 파이프라인

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing, housing_labels)
full_pipeline_with_predictor.predict(some_data)

## joblib을 사용한 모델 저장

In [ ]:
my_model = full_pipeline_with_predictor

In [ ]:
import joblib
# from sklearn.externals import joblib
joblib.dump(my_model, "my_model.pkl") # DIFF
#...
my_model_loaded = joblib.load("my_model.pkl") # DIFF

## RandomizedSearchCV을 위한 Scipy 분포 함수

In [ ]:
from scipy.stats import geom, expon
geom_distrib=geom(0.5).rvs(10000, random_state=42)
expon_distrib=expon(scale=1).rvs(10000, random_state=42)
plt.hist(geom_distrib, bins=50)
plt.show()
plt.hist(expon_distrib, bins=50)
plt.show()